In [46]:
import discogs_client

In [47]:
def read_user_token():
    try:
        with open("user_token.env", "r") as file:
            content = file.read()
            return content
    except FileNotFoundError:
        return {}
        
d = discogs_client.Client('dardok_app/0.1', user_token='{}'.format(read_user_token().rstrip()))

In [48]:
label_ids = "589255"
min_average_value = 4.75
min_threshold_value = 4.5

In [49]:
# COMMON FUNCTIONS

def get_labels_ids_list(label_ids):
    """ A function to convert elements to integers """
    label_id_list = [int(label_id) for label_id in label_ids.split(',')]
    return label_id_list

def get_filtered_release_lists_from_label(label, min_average, min_threshold=None):
    """ A function that returns filtered releases from label """
    all_releases = get_all_releases(label)

    filtered_release_list_1 = filter(lambda release: filter_1(release, min_average), all_releases)

    filtered_release_list_1 = list(filtered_release_list_1)
    
    print()
    print("filtered_release_list_1: {} elements".format(len(filtered_release_list_1)))
    for item in filtered_release_list_1:
        print(item)
    
    unique_filtered_release_list_1 = remove_duplicates(filtered_release_list_1)

    print()
    print("unique_filtered_release_list_1: {} elements".format(len(unique_filtered_release_list_1)))
    for item in unique_filtered_release_list_1:
        print(item)
    
    temp_releases = [item for item in all_releases if item not in unique_filtered_release_list_1]

    print()
    print("temp_releases: {} elements".format(len(temp_releases)))
    for item in temp_releases:
        print(item)
    
    filtered_release_list_2 = filter(lambda release: filter_2(release, min_average, min_threshold), temp_releases)

    filtered_release_list_2 = list(filtered_release_list_2)
    
    return filtered_release_list_1, filtered_release_list_2

def filter_1(release, min_average):
    """ A function to define a filter """
    return release.community.rating.average >= min_average and release.formats[0]['name'] == 'Vinyl' and 'Electronic' in release.genres

def filter_2(release, min_average, min_threshold):
    """ A function to define a filter """
    return min_threshold < release.community.rating.average < min_average and release.formats[0]['name'] == 'Vinyl' and 'Electronic' in release.genres

def get_all_releases(label):
    """ A function to get all releases from label """
    all_releases = []
    print()
    print("Processing label: {} {}...".format(label.name, label.url))
    for k in range(1, label.releases.pages + 1):
        print()
        print("Page: {}/{}".format(k, label.releases.pages))
        print(label.releases.page(k))
        for item in label.releases.page(k):
            all_releases.append(d.release(item.id))
    print()
    print("{} releases for label {}".format(len(all_releases), label.name))
    return all_releases

def func_year(e):
    """ A function that returns the 'year' value """
    return e.year

def remove_duplicates(input_list):
    """ A function to remove duplicates from a release input_list by id """
    unique_list = []
    seen_ids = set()

    for item in input_list:
        item_id = item.id
        if item_id not in seen_ids:
            seen_ids.add(item_id)
            unique_list.append(item)

    return unique_list

def get_artists(release):
    """ A function to return an artists list from release """
    artists = []
    for artist in release.artists:
        artists.append(artist.name)
    return artists

def get_labels(release):
    """ A function to return an labels list from release """
    labels = []
    for label in release.labels:
        labels.append(label.name)
    return labels

def write_output_html(label, release_list, range):
    """ A function to write output as HTML """

    if range == "higher":
        htmlString = " \
            <html>\n<head>\n<title>Filtered Releases for label {} - id: {}</title>\n</head>\n \
            <body>\n \
                <h1>Filtered Releases for label {} - id: {}</h1>\n \
                <h2>Url: <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a><h2>\n \
                <br> \
                HIGHER RANGE \
                <br> \
                average community rating >= <span style='color: #ff0000'>{}</span> \n \
                <br> \
                Number of results: {} \
            ".format(label.name, label.id, label.name, label.id, label.url, label.url, min_average_value, len(release_list))
        file_name = "{}-higher.html".format(label.url.lstrip('https://www.discogs.com/label/'))
    elif range == "middle":
        htmlString = " \
            <html>\n<head>\n<title>Filtered Releases for label {} - id: {}</title>\n</head>\n \
            <body>\n \
                <h1>Filtered Releases for label {} - id: {}</h1>\n \
                <h2>Url: <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a><h2>\n \
                <br> \
                MIDDLE RANGE \
                <br> \
                <span style='color: #ff0000'>{}</span> < average community rating < <span style='color: #ff0000'>{}</span> \n \
                <br> \
                Number of results: {} \
            ".format(label.name, label.id, label.name, label.id, label.url, label.url, min_threshold_value, min_average_value, len(release_list))
        file_name = "{}-middle.html".format(label.url.lstrip('https://www.discogs.com/label/'))


    htmlString += "<ul style='list-style: none;'>\n"
    k = 0
    for release in release_list:
        htmlString += "<br>\n"
        k += 1
        htmlString += "<span style='color: #5f5f5f'>Result {}/{}</span>\n".format(k, len(release_list)) 
        htmlString += "<li><span style='color: #9b9b9b'>catno:</span> {}</li>\n".format(release.data['labels'][0]['catno'])
        htmlString += "<li><span style='color: #9b9b9b'>Title:</span> {}</li>\n".format(release.title)
        artists = get_artists(release)
        htmlString += "<li><span style='color: #9b9b9b'>Artists:</span> {}</li>\n".format(artists)
        htmlString += "<li><span style='color: #9b9b9b'>Year:</span> {}</li>\n".format(release.year)
        labels = get_labels(release)
        htmlString += "<li><span style='color: #9b9b9b'>Labels:</span> {}</li>\n".format(labels)
        htmlString += "<li><span style='color: #9b9b9b'>Genres:</span> {}</li>\n".format(release.genres)
        htmlString += "<li><span style='color: #9b9b9b'>Styles:</span> {}</li>\n".format(release.styles)
        htmlString += "<li><span style='color: #9b9b9b'>Formats:</span> {}</li>\n".format(release.formats)
        htmlString += "<li><span style='color: #9b9b9b'>Community rating average/count:</span> <span style='color: #ff0000'>{}</span>/<span style='color: #ff8000'>{}</span></li>\n".format(release.community.rating.average, release.community.rating.count)
        htmlString += "<li><span style='color: #9b9b9b'>Url:</span> <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a></li>\n".format(release.url, release.url)

    htmlString += "</ul>\n"
    
    htmlString += "</body>\n \
                  </html>"
    
    with open("./outputHtml/{}".format(file_name), "w") as file:
        file.write(htmlString)

    print()
    print("Written HTML output file to ./outputHtml/{}".format(file_name))



In [50]:
labels_id_list = get_labels_ids_list(label_ids)
for label_id in labels_id_list:
    result_label = d.label(label_id)
    filtered_release_list_higher_rating, filtered_release_list_middle_rating = get_filtered_release_lists_from_label(result_label, min_average_value, min_threshold_value)
    
    print()
    print("{} releases for: average community rating >= {}".format(len(filtered_release_list_higher_rating), min_average_value))
    for release in filtered_release_list_higher_rating:
        print(release)

    print()
    print("{} releases for: {} < average community rating < {}".format(len(filtered_release_list_middle_rating), min_threshold_value, min_average_value))
    for release in filtered_release_list_middle_rating:
        print(release)

    # Process filtered_release_list_higher_rating
    filtered_release_list_higher_rating.sort(reverse=True, key=func_year)
    unique_filtered_release_list_higher_rating = remove_duplicates(filtered_release_list_higher_rating)
    write_output_html(result_label, unique_filtered_release_list_higher_rating, "higher")

    # Process filtered_release_list_middle_rating
    filtered_release_list_middle_rating.sort(reverse=True, key=func_year)
    unique_filtered_release_list_middle_rating = remove_duplicates(filtered_release_list_middle_rating)
    write_output_html(result_label, unique_filtered_release_list_middle_rating, "middle")
print()
print("Process finished")


Processing label: Nous'klaer Audio https://www.discogs.com/label/589255-Nousklaer-Audio...

Page: 1/3
[<Release 27960567 'Choral Feeling'>, <Release 27960567 'Choral Feeling'>, <Release 24828707 'Founds On Land'>, <Release 24828707 'Founds On Land'>, <Release 20517262 'Rift'>, <Release 20517262 'Rift'>, <Release 25693252 'Choral Feeling'>, <Release 25693252 'Choral Feeling'>, <Release 28216465 'Chiral'>, <Release 28216465 'Chiral'>, <Release 28242931 "Nous'klaer Audio X Draaimolen present Aura">, <Release 10838529 'Herds (Oceanic Remix)'>, <Release 12632258 'Emerald Island'>, <Release 13136445 'Gegek'>, <Release 16091652 'Orbu'>, <Release 22946090 'Gelgit'>, <Release 17674057 'Loving/Sight To Sea'>, <Release 8462402 'Loving/Sight To Sea'>, <Release 28216465 'Chiral'>, <Release 23215445 'May + Be'>, <Release 7474298 "Olympics '84 ">, <Release 26243321 'An Inclination'>, <Release 4904450 'Isms EP'>, <Release 6896600 'Isms EP'>, <Release 6486729 'Remixed By Oceanic And Mattheis'>, <Relea